<a href="https://colab.research.google.com/github/AdrianUrbanski/ADM/blob/main/Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 5.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617558 sha256=9ed1ec550a2032cac08dcc16af745afeb418b0c72957c8c62df81246563a387d
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [9]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import pickle
import os

from google.colab import drive

import pandas as pd
import numpy as np

from surprise import SVD
from surprise import NMF
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import dump
from surprise.accuracy import rmse
from surprise.accuracy import mae

# Preparing dataset

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ratings_df = pd.read_csv('/content/drive/My Drive/ratings.csv.gz', compression='gzip')[['bgg_user_name', 'bgg_id', 'bgg_user_rating']]

In [5]:
ratings_df = ratings_df[ratings_df['bgg_user_rating']>=1]

In [26]:
users = ratings_df['bgg_user_name'].unique()
np.random.shuffle(users)
train_size = int(0.7*users.shape[0])
train_df = ratings_df[ratings_df['bgg_user_name'].isin(users[:train_size])]
test_df = ratings_df[ratings_df['bgg_user_name'].isin(users[train_size:])]

In [93]:
def split_test(test_df, seed=42, frac=0.8):
    grouped = test_df.groupby(by='bgg_user_name')
    test_known = []
    test_unknown = []
    for user, df in grouped:
        df_size = df.shape[0]

        known_size = int(round(frac*df_size))
        known_indices = np.random.choice(df_size, known_size, replace=False)
        known_data = df.iloc[known_indices]
        test_known.append(known_data)

        unknown_indices = np.setdiff1d(np.arange(df_size), known_indices)
        unknown_data = df.iloc[unknown_indices]
        test_unknown.append(unknown_data)

    return pd.concat(test_known), pd.concat(test_unknown)

In [94]:
test_known, test_unknown = split_test(test_df)

In [96]:
test_unknown.shape

(1031434, 3)

In [97]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(train_df.append(test_known), reader)

# Tests